In [27]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession 
from pyspark.sql.functions import * 

import warnings
warnings.filterwarnings("ignore")

In [28]:
spark = SparkSession.builder.appName("Classifier").config('spark.executor.memory', '1G').getOrCreate()

In [29]:
user_profiles = spark.read.csv('data/user_profiles.csv', header=True, sep = ',',multiLine=True, escape = "\"",inferSchema =True)
politifact_tweets = spark.read.csv('data/politifact_tweets_processed.csv', header=True, sep = ',',multiLine=True, escape = "\"",inferSchema =True)
gossipcop_tweets = spark.read.csv('data/gossipcop_tweets_processed.csv', header=True, sep = ',',multiLine=True, escape = "\"",inferSchema =True) 

In [31]:
user_profiles = user_profiles.withColumnRenamed("created_at", "created_at_user")\
    .withColumnRenamed("verified", "verified_user").withColumnRenamed("followers_count", "followers_count_user")

In [32]:
politifact_tweets = politifact_tweets.withColumnRenamed("created_at", "created_at_tweet")\
    .withColumnRenamed("verified", "verified_y").withColumnRenamed("followers_count", "followers_count_tweet")
gossipcop_tweets = gossipcop_tweets.withColumnRenamed("created_at", "created_at_tweet")\
    .withColumnRenamed("verified", "verified_y").withColumnRenamed("followers_count", "followers_count_tweet")

In [33]:
politifact_with_users = politifact_tweets.join(user_profiles, on=['user_id'], how='left_outer')
gossipcop_with_users = gossipcop_tweets.join(user_profiles, on=['user_id'], how='left_outer')

In [37]:
politifact_with_users = politifact_with_users.withColumn('created_at_tweet', to_timestamp(col('created_at_tweet'), 'yyyy-MM-dd HH:mm:ss'))
politifact_with_users = politifact_with_users.withColumn('created_at_tweet', politifact_with_users.created_at_tweet.cast("long"))


In [38]:
gossipcop_with_users = gossipcop_with_users.withColumn('created_at_tweet', to_timestamp(col('created_at_tweet'), 'yyyy-MM-dd HH:mm:ss'))
gossipcop_with_users = gossipcop_with_users.withColumn('created_at_tweet', gossipcop_with_users.created_at_tweet.cast("long"))

In [39]:
politifact_with_users.toPandas().to_csv('politifact_with_users.csv')
gossipcop_with_users.toPandas().to_csv('gossipcop_with_users.csv')